In [37]:
import pandas as pd
import numpy as np

In [38]:
df = pd.read_csv('./dataset/online_retail_cleaned.csv')

In [39]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [40]:
df['Revenue'] = df['Price'] * df['Quantity']

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805620 entries, 0 to 805619
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805620 non-null  int64         
 1   StockCode    805620 non-null  object        
 2   Description  805620 non-null  object        
 3   Quantity     805620 non-null  int64         
 4   InvoiceDate  805620 non-null  datetime64[ns]
 5   Price        805620 non-null  float64       
 6   Customer ID  805620 non-null  int64         
 7   Country      805620 non-null  object        
 8   Revenue      805620 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 55.3+ MB


In [42]:
df['Quarter'] = df['InvoiceDate'].dt.to_period('Q')

In [43]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,Quarter
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,2009Q4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009Q4
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009Q4
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,2009Q4
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,2009Q4


In [44]:
revenue_by_quarter = df.groupby(['Customer ID', 'Quarter'])['Revenue'].sum().reset_index()
revenue_by_quarter

,Customer ID,Quarter,Revenue
0,12346,2009Q4,113.50
1,12346,2010Q1,117.05
2,12346,2010Q2,142.31
3,12346,2011Q1,77183.60
4,12347,2010Q4,2035.11
...,...,...,...
18021,18287,2010Q2,1071.61
18022,18287,2010Q3,892.60
18023,18287,2010Q4,381.50
18024,18287,2011Q2,765.28


In [45]:
all_quarters = pd.MultiIndex.from_product([df['Customer ID'].unique(), df['Quarter'].unique()], names=['Customer ID', 'Quarter'])
all_quarters_df = pd.DataFrame(index=all_quarters).reset_index()

revenue_by_quarter = df.groupby(['Customer ID', 'Quarter'])['Revenue'].sum().reset_index()

merged_df = pd.merge(all_quarters_df, revenue_by_quarter, on=['Customer ID', 'Quarter'], how='left')

merged_df['Revenue'].fillna(0, inplace=True)

In [47]:
merged_df['Quarter End'] = merged_df['Quarter'].dt.end_time

In [48]:
merged_df

,Customer ID,Quarter,Revenue,Quarter End
0,13085,2009Q4,1096.80,2009-12-31 23:59:59.999999999
1,13085,2010Q1,920.40,2010-03-31 23:59:59.999999999
2,13085,2010Q2,0.00,2010-06-30 23:59:59.999999999
3,13085,2010Q3,0.00,2010-09-30 23:59:59.999999999
4,13085,2010Q4,0.00,2010-12-31 23:59:59.999999999
...,...,...,...,...
52924,12713,2010Q4,0.00,2010-12-31 23:59:59.999999999
52925,12713,2011Q1,0.00,2011-03-31 23:59:59.999999999
52926,12713,2011Q2,0.00,2011-06-30 23:59:59.999999999
52927,12713,2011Q3,0.00,2011-09-30 23:59:59.999999999


In [59]:
df.groupby(['Customer ID', 'Quarter'])['InvoiceDate'].max().reset_index()

,Customer ID,Quarter,InvoiceDate
0,12346,2009Q4,2009-12-18 10:55:00
1,12346,2010Q1,2010-03-02 13:08:00
2,12346,2010Q2,2010-06-28 13:53:00
3,12346,2011Q1,2011-01-18 10:01:00
4,12347,2010Q4,2010-12-07 14:57:00
...,...,...,...
18021,18287,2010Q2,2010-05-17 11:55:00
18022,18287,2010Q3,2010-09-21 12:19:00
18023,18287,2010Q4,2010-11-22 11:51:00
18024,18287,2011Q2,2011-05-22 10:39:00


In [61]:
df['Quarter'] = df['InvoiceDate'].dt.to_period('Q')
temp = df.groupby(['Customer ID', 'Quarter'])['InvoiceDate'].max().reset_index()
temp.rename(columns={'InvoiceDate': 'Last Seen (days)'}, inplace=True)
temp['Quarter End'] = temp['Quarter'].dt.end_time
temp['Recency'] = (temp['Quarter End'] - temp['Last Seen (days)']).dt.days
merged_df = pd.merge(merged_df, temp[['Customer ID', 'Quarter', 'Recency']], on=['Customer ID', 'Quarter'], how='left')

In [62]:
merged_df

,Customer ID,Quarter,Revenue,Quarter End,Recency
0,13085,2009Q4,1096.80,2009-12-31 23:59:59.999999999,28.0
1,13085,2010Q1,920.40,2010-03-31 23:59:59.999999999,61.0
2,13085,2010Q2,0.00,2010-06-30 23:59:59.999999999,NaN
3,13085,2010Q3,0.00,2010-09-30 23:59:59.999999999,NaN
4,13085,2010Q4,0.00,2010-12-31 23:59:59.999999999,NaN
...,...,...,...,...,...
52924,12713,2010Q4,0.00,2010-12-31 23:59:59.999999999,NaN
52925,12713,2011Q1,0.00,2011-03-31 23:59:59.999999999,NaN
52926,12713,2011Q2,0.00,2011-06-30 23:59:59.999999999,NaN
52927,12713,2011Q3,0.00,2011-09-30 23:59:59.999999999,NaN


In [83]:
merged_df.groupby('Quarter')['Revenue'].quantile(0.8).reset_index()


,Quarter,Revenue
0,2009Q4,0.00
1,2010Q1,302.15
2,2010Q2,351.40
3,2010Q3,367.65
4,2010Q4,595.90
5,2011Q1,284.97
6,2011Q2,331.46
7,2011Q3,400.37
8,2011Q4,492.50


In [95]:
non_zero_revenues = merged_df[merged_df['Revenue'] > 0]['Revenue']
percentiles = non_zero_revenues.quantile([0.8, 0.6, 0.4, 0.2]).to_dict()

In [96]:
def assign_score(row):
    if row['Revenue'] == 0:
        return 0
    elif row['Revenue'] > percentiles[0.8]:
        return 5
    elif row['Revenue'] > percentiles[0.6]:
        return 4
    elif row['Revenue'] > percentiles[0.4]:
        return 3
    elif row['Revenue'] > percentiles[0.2]:
        return 2
    else:
        return 1

In [97]:
merged_df['Revenue Score'] = merged_df.apply(assign_score, axis=1)

In [103]:
merged_df.to_excel('./dataset/quarterly_revenue.xlsx')



In [104]:
merged_df

,Customer ID,Quarter,Revenue,Quarter End,Recency,Revenue Score
0,13085,2009Q4,1096.80,2009-12-31 23:59:59.999999999,28.0,5
1,13085,2010Q1,920.40,2010-03-31 23:59:59.999999999,61.0,4
2,13085,2010Q2,0.00,2010-06-30 23:59:59.999999999,NaN,0
3,13085,2010Q3,0.00,2010-09-30 23:59:59.999999999,NaN,0
4,13085,2010Q4,0.00,2010-12-31 23:59:59.999999999,NaN,0
...,...,...,...,...,...,...
52924,12713,2010Q4,0.00,2010-12-31 23:59:59.999999999,NaN,0
52925,12713,2011Q1,0.00,2011-03-31 23:59:59.999999999,NaN,0
52926,12713,2011Q2,0.00,2011-06-30 23:59:59.999999999,NaN,0
52927,12713,2011Q3,0.00,2011-09-30 23:59:59.999999999,NaN,0


In [134]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,Quarter
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.40,2009Q4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.00,2009Q4
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.00,2009Q4
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.80,2009Q4
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.00,2009Q4
...,...,...,...,...,...,...,...,...,...,...
805615,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60,2011Q4
805616,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60,2011Q4
805617,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60,2011Q4
805618,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,14.85,2011Q4


In [137]:
merged_df['Frequency'] = df.groupby('Customer ID')['Quarter'].count()
merged_df.head(10)



,Customer ID,Quarter,Revenue,Quarter End,Recency,Revenue Score,Frequency
0,13085,2009Q4,1096.80,2009-12-31 23:59:59.999999999,28.0,5,NaN
1,13085,2010Q1,920.40,2010-03-31 23:59:59.999999999,61.0,4,NaN
2,13085,2010Q2,0.00,2010-06-30 23:59:59.999999999,NaN,0,NaN
3,13085,2010Q3,0.00,2010-09-30 23:59:59.999999999,NaN,0,NaN
4,13085,2010Q4,0.00,2010-12-31 23:59:59.999999999,NaN,0,NaN
5,13085,2011Q1,278.10,2011-03-31 23:59:59.999999999,42.0,2,NaN
6,13085,2011Q2,0.00,2011-06-30 23:59:59.999999999,NaN,0,NaN
7,13085,2011Q3,137.98,2011-09-30 23:59:59.999999999,87.0,1,NaN
8,13085,2011Q4,0.00,2011-12-31 23:59:59.999999999,NaN,0,NaN
9,13078,2009Q4,2319.03,2009-12-31 23:59:59.999999999,11.0,5,NaN


In [131]:
temp = merged_df.copy()
temp['Next Quarter'] = temp['Quarter'].shift(-1)
temp['Next Quarter Revenue Score'] = temp['Revenue Score'].shift(-1)
temp = temp[temp['Quarter'] != '2011Q4']
temp.head(10)


,Customer ID,Quarter,Revenue,Quarter End,Recency,Revenue Score,Next Quarter,Next Quarter Revenue Score
0,13085,2009Q4,1096.80,2009-12-31 23:59:59.999999999,28.0,5,2010Q1,4.0
1,13085,2010Q1,920.40,2010-03-31 23:59:59.999999999,61.0,4,2010Q2,0.0
2,13085,2010Q2,0.00,2010-06-30 23:59:59.999999999,NaN,0,2010Q3,0.0
3,13085,2010Q3,0.00,2010-09-30 23:59:59.999999999,NaN,0,2010Q4,0.0
4,13085,2010Q4,0.00,2010-12-31 23:59:59.999999999,NaN,0,2011Q1,2.0
5,13085,2011Q1,278.10,2011-03-31 23:59:59.999999999,42.0,2,2011Q2,0.0
6,13085,2011Q2,0.00,2011-06-30 23:59:59.999999999,NaN,0,2011Q3,1.0
7,13085,2011Q3,137.98,2011-09-30 23:59:59.999999999,87.0,1,2011Q4,0.0
9,13078,2009Q4,2319.03,2009-12-31 23:59:59.999999999,11.0,5,2010Q1,5.0
10,13078,2010Q1,3148.64,2010-03-31 23:59:59.999999999,7.0,5,2010Q2,5.0


In [132]:
temp.drop(['Recency', 'Quarter End', 'Revenue'], axis=1, inplace=True)
temp['Next Quarter Revenue Score'] = temp['Next Quarter Revenue Score'].astype(int)
temp.to_csv('./dataset/quarterly_transition01.csv', index=False)



In [129]:
temp = temp.groupby(['Quarter', 'Next Quarter', 'Revenue Score', 'Next Quarter Revenue Score']).count().reset_index()
temp.drop(['Customer ID', 'Revenue', 'Recency'], axis=1, inplace=True)
temp.rename(columns={'Quarter End': 'Count'}, inplace=True)
temp['Next Quarter Revenue Score'] = temp['Next Quarter Revenue Score'].astype(int)
temp


,Quarter,Next Quarter,Revenue Score,Next Quarter Revenue Score,Count
0,2009Q4,2010Q1,0,0,3726
1,2009Q4,2010Q1,0,1,298
2,2009Q4,2010Q1,0,2,266
3,2009Q4,2010Q1,0,3,266
4,2009Q4,2010Q1,0,4,245
...,...,...,...,...,...
283,2011Q3,2011Q4,5,1,4
284,2011Q3,2011Q4,5,2,13
285,2011Q3,2011Q4,5,3,39
286,2011Q3,2011Q4,5,4,73


In [130]:
temp.to_csv('./dataset/quarterly_transition.csv')



In [138]:
temp3 = df.groupby(['Customer ID', 'Quarter'])['InvoiceDate'].count().reset_index()
temp3 = pd.merge(merged_df, temp[['Customer ID', 'Quarter', 'Recency']], on=['Customer ID', 'Quarter'], how='left')

,Customer ID,Quarter,InvoiceDate
0,12346,2009Q4,5
1,12346,2010Q1,9
2,12346,2010Q2,19
3,12346,2011Q1,1
4,12347,2010Q4,102
...,...,...,...
18021,18287,2010Q2,54
18022,18287,2010Q3,21
18023,18287,2010Q4,10
18024,18287,2011Q2,29


In [ ]:
df['Quarter'] = df['InvoiceDate'].dt.to_period('Q')
temp = df.groupby(['Customer ID', 'Quarter'])['InvoiceDate'].max().reset_index()
temp.rename(columns={'InvoiceDate': 'Last Seen (days)'}, inplace=True)
temp['Quarter End'] = temp['Quarter'].dt.end_time
temp['Recency'] = (temp['Quarter End'] - temp['Last Seen (days)']).dt.days
merged_df = pd.merge(merged_df, temp[['Customer ID', 'Quarter', 'Recency']], on=['Customer ID', 'Quarter'], how='left')